In [30]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from delta import *
import os
import pandas as pd
import json

In [37]:
spark = SparkSession.builder \
    .appName("CSGO_Matches") \
    .master('local[*]') \
    .getOrCreate()

In [2]:
spark = SparkSession.builder \
    .config('spark.jars', 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-j-8.0.31.jar') \
    .appName("CSGO_Matches") \
    .master('local[*]') \
    .getOrCreate()

In [2]:
builder = SparkSession.builder \
    .master('local[*]') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .appName("CSGO_Matches")

spark = configure_spark_with_delta_pip(builder) \
        .getOrCreate()

In [5]:
df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/oficina") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "veiculo") \
    .option("user", "root").option("password", "root").load()
df.show()

NameError: name 'spark' is not defined

In [1]:
import requests

In [4]:
gameId = 100015682
api_key = "dedaad020f8d43b2aafcb7d1460a1f4b"
url = f"https://api.sportsdata.io/v3/csgo/stats/json/BoxScore/{gameId}"
response = requests.get(url, headers={"Ocp-Apim-Subscription-Key": f"{api_key}"})
response.json()

[{'Game': {'GameId': 100015682,
   'RoundId': 100002653,
   'Season': 2022,
   'SeasonType': 3,
   'Group': None,
   'TeamAId': 100000205,
   'TeamBId': 100000502,
   'VenueId': None,
   'Day': '2022-09-28T00:00:00',
   'DateTime': '2022-09-28T13:30:00',
   'Status': 'Final',
   'Week': None,
   'BestOf': '3',
   'Winner': 'Scrambled',
   'VenueType': None,
   'TeamAKey': 'FUR',
   'TeamAName': 'FURIA Esports',
   'TeamAScore': 0,
   'TeamBKey': 'OUT',
   'TeamBName': 'Outsiders',
   'TeamBScore': 0,
   'TeamAMoneyLine': None,
   'TeamBMoneyLine': None,
   'DrawMoneyLine': None,
   'PointSpread': None,
   'TeamAPointSpreadPayout': None,
   'TeamBPointSpreadPayout': None,
   'Updated': '2022-09-28T13:07:27',
   'UpdatedUtc': '2022-09-28T17:07:27',
   'IsClosed': True},
  'Maps': [{'Number': 1,
    'Name': 'de_ancient',
    'Status': 'Scheduled',
    'CurrentRound': 0,
    'TeamAScore': 0,
    'TeamBScore': 0,
    'Leaderboards': [{'PlayerId': 100001797,
      'TeamId': 100000205,
      

In [4]:
def get_games_ids():
  df_history = spark.read.format("delta").load("spark-warehouse/bronze.db/csgo_match_history")
  try:
    df_proceeded = spark.read.parquet("raw/games_proceeded")
    df_join = df_history.join(df_proceeded, "gameId", "left").filter("path is null").select("gameId")
    return df_join
  except:
    return df_history.select("gameId")

In [5]:
df_history = spark.read.format("delta").load("spark-warehouse/bronze.db/csgo_match_history")
df_history

DataFrame[GameId: int, RoundId: int, Season: int, SeasonType: int, Group: string, TeamAId: int, TeamBId: int, VenueId: int, Day: string, DateTime: string, Status: string, Week: int, BestOf: string, Winner: string, VenueType: string, TeamAKey: string, TeamAName: string, TeamAScore: int, TeamBKey: string, TeamBName: string, TeamBScore: int, TeamAMoneyLine: int, TeamBMoneyLine: int, DrawMoneyLine: int, PointSpread: decimal(10,0), TeamAPointSpreadPayout: int, TeamBPointSpreadPayout: int, Updated: string, UpdatedUtc: string, IsClosed: boolean, row_number: int]

In [6]:
df = get_games_ids()
df.show()

+---------+
|   gameId|
+---------+
|100015581|
|100015582|
|100015583|
|100015584|
|100015585|
|100015586|
|100015587|
|100015588|
|100015589|
|100015590|
|100015591|
|100015592|
|100015679|
|100015680|
|100015681|
|100015682|
|100015683|
|100015684|
|100015685|
|100015686|
+---------+
only showing top 20 rows



In [40]:
df = pd.read_parquet("spark-warehouse/bronze.db/csgo_match_history")
df

,GameId,RoundId,Season,SeasonType,Group,TeamAId,TeamBId,VenueId,Day,DateTime,...,TeamBScore,TeamAMoneyLine,TeamBMoneyLine,DrawMoneyLine,PointSpread,TeamAPointSpreadPayout,TeamBPointSpreadPayout,Updated,UpdatedUtc,IsClosed
0,100015581,100002639,2022,1,Group D,100000081,100000522,NaN,2022-09-22T00:00:00,2022-09-22T10:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-22T08:56:23,2022-09-22T12:56:23,True
1,100015582,100002639,2022,1,Group D,100000267,100000205,NaN,2022-09-22T00:00:00,2022-09-22T14:00:00,...,1.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-22T12:21:28,2022-09-22T16:21:28,True
2,100015583,100002639,2022,1,Group D,100000098,100000234,NaN,2022-09-22T00:00:00,2022-09-22T17:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-22T15:42:22,2022-09-22T19:42:22,True
3,100015584,100002639,2022,1,Group D,100000081,100000267,NaN,2022-09-23T00:00:00,2022-09-23T10:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-23T08:28:27,2022-09-23T12:28:27,True
4,100015585,100002639,2022,1,Group D,100000522,100000234,NaN,2022-09-23T00:00:00,2022-09-23T14:00:00,...,1.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-23T12:07:21,2022-09-23T16:07:21,True
5,100015586,100002639,2022,1,Group D,100000098,100000205,NaN,2022-09-23T00:00:00,2022-09-23T17:30:00,...,1.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-23T16:42:28,2022-09-23T20:42:28,True
6,100015587,100002639,2022,1,Group D,100000267,100000234,NaN,2022-09-24T00:00:00,2022-09-24T10:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-24T08:49:23,2022-09-24T12:49:23,True
7,100015588,100002639,2022,1,Group D,100000098,100000522,NaN,2022-09-24T00:00:00,2022-09-24T14:00:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-24T12:28:19,2022-09-24T16:28:19,True
8,100015589,100002639,2022,1,Group D,100000081,100000205,NaN,2022-09-24T00:00:00,2022-09-24T17:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-24T15:49:23,2022-09-24T19:49:23,True
9,100015590,100002639,2022,1,Group D,100000081,100000098,NaN,2022-09-25T00:00:00,2022-09-25T17:30:00,...,0.0,NaN,NaN,NaN,None,NaN,NaN,2022-09-25T16:49:29,2022-09-25T20:49:29,True


In [36]:
df_proceeded = spark.read.parquet("raw/games_proceeded")
df_maps = spark.read.parquet("raw/tb_maps")
df_join = df_proceeded.join(df_maps, "gameId", "left")
df_join.show()

+---------+--------------------+------+-----------+----------+------------+----------+----------+--------------+--------------+--------+--------+
|   gameId|                path|Number|       Name|    Status|CurrentRound|TeamAScore|TeamBScore|     TeamAName|     TeamBName|TeamAKey|TeamBKey|
+---------+--------------------+------+-----------+----------+------------+----------+----------+--------------+--------------+--------+--------+
|100015748|raw/games_landing...|     1|de_overpass| Scheduled|           0|         0|         0|          MIBR|      Team One|     MIB|     ONE|
|100015688|raw/games_landing...|     1|  de_mirage| Scheduled|           0|         0|         0| Team Vitality|    G2 Esports|     VIT|     G2E|
|100015582|raw/games_landing...|     1| de_ancient| Scheduled|           0|         0|         0|Movistar Rider| FURIA Esports|     MOV|     FUR|
|100015685|raw/games_landing...|     1|   de_dust2| Scheduled|           0|         0|         0|    G2 Esports| Natus Vince

In [27]:
df = pd.read_parquet("raw/tb_maps/")
df

,Number,Name,Status,CurrentRound,TeamAScore,TeamBScore,GameId,TeamAName,TeamBName,TeamAKey,TeamBKey
0,1,de_dust2,Scheduled,0,0,0,100015581,Cloud9,Eternal Fire,CLO,ETE


In [25]:
df = pd.read_parquet("raw/tb_leaderboards/")
df

,PlayerId,TeamId,Name,MatchName,Team,IsClosed,GameId,OpponentId,Opponent,Day,...,Kast,Rating,EntryKills,QuadKills,Aces,Clutch1v2s,Clutch1v3s,Clutch1v4s,Clutch1v5s,MapName
0,100001211,100000081,Abay Khasenov,HObbit,Cloud9,True,100015581,100000522,Eternal Fire,2022-09-22T00:00:00,...,46.599998,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
1,100001376,100000081,Sergey Rykhtorov,Ax1Le,Cloud9,True,100015581,100000522,Eternal Fire,2022-09-22T00:00:00,...,44.099998,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
2,100002289,100000081,Timofey Yakushin,interz,Cloud9,True,100015581,100000522,Eternal Fire,2022-09-22T00:00:00,...,53.900002,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
3,100002291,100000081,Dmitry Sokolov,sh1ro,Cloud9,True,100015581,100000522,Eternal Fire,2022-09-22T00:00:00,...,51.400002,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
4,100002292,100000081,Vladislav Gorshkov,nafany,Cloud9,True,100015581,100000522,Eternal Fire,2022-09-22T00:00:00,...,41.599998,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
5,100001028,100000522,Can Dörtkardeş,XANTARES,Eternal Fire,True,100015581,100000081,Cloud9,2022-09-22T00:00:00,...,53.900002,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
6,100001031,100000522,Ahmet Karahoca,paz,Eternal Fire,True,100015581,100000081,Cloud9,2022-09-22T00:00:00,...,46.599998,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
7,100001032,100000522,Engin Kupeli,MAJ3R,Eternal Fire,True,100015581,100000081,Cloud9,2022-09-22T00:00:00,...,39.200001,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
8,100001208,100000522,Özgür Eker,woxic,Eternal Fire,True,100015581,100000081,Cloud9,2022-09-22T00:00:00,...,41.599998,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2
9,100003399,100000522,Ömer Karataş,imoRR,Eternal Fire,True,100015581,100000081,Cloud9,2022-09-22T00:00:00,...,39.200001,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de_dust2


In [7]:
df = pd.read_parquet("raw/games_proceeded/")
df

,gameId,path
0,100015748,raw/games_landing/100015748.json
1,100015688,raw/games_landing/100015688.json
2,100015582,raw/games_landing/100015582.json
3,100015685,raw/games_landing/100015685.json
4,100015586,raw/games_landing/100015586.json
5,100015747,raw/games_landing/100015747.json
6,100015583,raw/games_landing/100015583.json
7,100015680,raw/games_landing/100015680.json
8,100015591,raw/games_landing/100015591.json
9,100015684,raw/games_landing/100015684.json


In [18]:
json.load(path)

AttributeError: 'str' object has no attribute 'read'

In [21]:
path = "raw/games_landing/100015584.json"
with open(path, 'r') as file:
    print(type(json.loads(file.read())))

<class 'list'>


In [22]:
path = "raw/games_landing/100015584.json"
with open(path, 'r') as file:
    print(json.load(file))

[{'Game': {'GameId': 100015584, 'RoundId': 100002639, 'Season': 2022, 'SeasonType': 1, 'Group': 'Group D', 'TeamAId': 100000081, 'TeamBId': 100000267, 'VenueId': None, 'Day': '2022-09-23T00:00:00', 'DateTime': '2022-09-23T10:30:00', 'Status': 'Final', 'Week': None, 'BestOf': '3', 'Winner': 'Scrambled', 'VenueType': None, 'TeamAKey': 'CLO', 'TeamAName': 'Cloud9', 'TeamAScore': 4, 'TeamBKey': 'MOV', 'TeamBName': 'Movistar Rider', 'TeamBScore': 0, 'TeamAMoneyLine': None, 'TeamBMoneyLine': None, 'DrawMoneyLine': None, 'PointSpread': None, 'TeamAPointSpreadPayout': None, 'TeamBPointSpreadPayout': None, 'Updated': '2022-09-23T08:28:27', 'UpdatedUtc': '2022-09-23T12:28:27', 'IsClosed': True}, 'Maps': [{'Number': 1, 'Name': 'de_inferno', 'Status': 'Scheduled', 'CurrentRound': 0, 'TeamAScore': 0, 'TeamBScore': 0, 'Leaderboards': [{'PlayerId': 100001211, 'TeamId': 100000081, 'Name': 'Abay Khasenov', 'MatchName': 'HObbit', 'Team': 'Cloud9', 'IsClosed': True, 'GameId': 100015584, 'OpponentId': 100

In [24]:
from  process_game_details import *
game_ids = get_ids()
for i in game_ids.collect():
    process_game(i[0])

In [32]:
spark.stop()